In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
dropout=0.2

cuda


In [2]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '#', '&', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '5', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']


In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) } 
encode = lambda s: [string_to_int[c] for c in s] 
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([39, 56, 67, 59, 52, 17,  1, 39, 55, 52,  1, 42, 62, 61, 51, 52, 65, 53,
        68, 59,  1, 42, 56, 73, 48, 65, 51,  1, 62, 53,  1, 34, 73,  0,  0,  0,
        20, 68, 67, 55, 62, 65, 17,  1, 31, 10,  1, 25, 65, 48, 61, 58,  1, 21,
        48, 68, 60,  0,  0, 37, 52, 59, 52, 48, 66, 52,  1, 51, 48, 67, 52, 17,
         1, 25, 52, 49, 65, 68, 48, 65, 72,  1, 12,  8,  1, 12, 16, 16, 14,  1,
        46, 52, 21, 62, 62, 58,  1,  3, 15, 15])


In [4]:
n = int(.8*len(data))
train_data = data[:n]
test_data = data[n:]

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t] 
    print(f'when input is {context} target is {target}')

when input is tensor([39]) target is 56
when input is tensor([39, 56]) target is 67
when input is tensor([39, 56, 67]) target is 59
when input is tensor([39, 56, 67, 59]) target is 52
when input is tensor([39, 56, 67, 59, 52]) target is 17
when input is tensor([39, 56, 67, 59, 52, 17]) target is 1
when input is tensor([39, 56, 67, 59, 52, 17,  1]) target is 39
when input is tensor([39, 56, 67, 59, 52, 17,  1, 39]) target is 55


In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

tensor([ 68419,   9177, 125941, 133306])
inputs:
tensor([[59, 62, 69, 52, 59, 72,  1, 50],
        [65, 62, 70, 56, 61, 54,  1, 49],
        [67, 55, 48, 61,  1, 52, 69, 52],
        [ 1, 66, 55, 52,  1, 50, 48, 59]], device='cuda:0')
targets:
tensor([[62, 69, 52, 59, 72,  1, 50, 62],
        [62, 70, 56, 61, 54,  1, 49, 56],
        [55, 48, 61,  1, 52, 69, 52, 65],
        [66, 55, 52,  1, 50, 48, 59, 59]], device='cuda:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C) #focus on last time step
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device) #torch.long int64
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


bfN?iuabUReyHNjRojMy0WKfEVa3Ag3
Xi’lN(iW’P 5,mitKuvZ’NrE2JH3tx’Xo,(Eu?zLiUtomHZ2“J)“Q!)hN’hS2?C5“dPJ& e33rD(O9TBnJ“*2s!R)jB&&“”Fw]kg.h!Eu#
#d.UvbjnTMsn

x0:G—0![3“Q”0)x[S2*yJO5sAt-TJH0ae33*fj;ct“‘Y2a)BwLJv‘j lsI(B”tK.&e;)k:G“”H3U5M]3(i“).
5VGi”Edfc g3es!N3nJA“Q*e*,F*CTY*EwQ!qE#Qy*-T’ngZO*#dXom:oI&WK“)Uaaw()A(LGZH:nr
NvYS1dDYc5*u?#“)t0,#)CZtjXT3.jWtG’Lxv YS]k0,l—LDYZ’Eu10i*WK *Cht#ey‘:!l?);L*s 2Hq
itSju ?.FaHZQP[)n?CKW!OSLsz!1yJH:Uaz!IHP[DkShUvsYknrWy gM“PCh.(Li5Zr!)mZDYc,wQXorbGe
CChcO.&d*(PMyke


In [8]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([ 70273, 102689, 131334, 157630])
tensor([15911, 26767,   544, 76605])
tensor([ 22333,  52606,  37739, 165154])
tensor([139265,  79330,  70463, 153193])
tensor([ 55964, 112343,  79258, 142770])
tensor([ 52106,  13109, 120428, 152261])
tensor([ 10834,  70897, 129755,  52632])
tensor([112433,  35467, 104174, 107514])
tensor([12488, 96978, 38074, 56574])
tensor([ 86395, 138432,  74317,  32385])
tensor([166405, 148092,  35715, 165435])
tensor([150806,  58645,  23684,   6540])
tensor([135107,  51151,  80995, 112920])
tensor([ 30735, 130894,  72592,  96621])
tensor([129438,  41276,  67912, 119887])
tensor([150098, 141515,  50723,  70442])
tensor([151652,  22041,  24343,  76874])
tensor([10435, 90978, 77210, 53869])
tensor([106647, 115564, 128193, 147353])
tensor([103308, 126756, 116149,  35649])
tensor([106856,   8964,  43853, 153400])
tensor([ 35318, 157386,  91421,  27256])
tensor([ 92988,  23786, 157921,  69636])
tensor([151341, 166322, 101833,  42503])
tensor([ 78723,    646,  239



need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them

    Mean Squared Error (MSE): MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
    Gradient Descent (GD): is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
    Momentum: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
    RMSprop: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
    Adam: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
    AdamW: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)